In [1]:
import time
import math
import sys
import numpy as np
from os.path import abspath, join
from scipy.optimize import minimize

from dcpyps import dcio, dataset, mechanism
from dcprogs.likelihood import Log10Likelihood, set_nthreads
print set_nthreads(2)

2


Load data

In [2]:
datadir = abspath("../../dc-pyps/dcpyps/samples/glydemo")
scnfiles = [join(datadir, u) for u in ["A-10.scn", "B-30.scn", "C-100.scn", "D-1000.scn"]]
tres = [0.000030, 0.000030, 0.000030, 0.000030]
tcrit = [0.004, -1, -0.06, -0.02]
conc = [10e-6, 30e-6, 100e-6, 1000e-6]

recs = []
bursts = []
for i in range(len(scnfiles)):
    rec = dataset.SCRecord([scnfiles[i]], conc[i], tres[i], tcrit[i])
    rec.record_type = 'recorded'
    recs.append(rec)
    bursts.append(rec.bursts.intervals())
    rec.printout()



 Data loaded from file: /Users/mdavezac/usr/src/dcprogs/dc-pyps/dcpyps/samples/glydemo/A-10.scn
Concentration of agonist = 10.000 microMolar
Resolution for HJC calculations = 30.0 microseconds
Critical gap length to define end of group (tcrit) = 4.000 milliseconds
	(defined so that all openings in a group prob come from same channel)
Initial and final vectors for bursts calculated asin Colquhoun, Hawkes & Srodzinski, (1996, eqs 5.8, 5.11).

Number of resolved intervals = 14553
Number of resolved periods = 12322

Number of open periods = 6161
Mean and SD of open periods = 1.288416392 +/- 1.982357560 ms
Range of open periods from 0.030309817 ms to 29.300384013 ms

Number of shut intervals = 6161
Mean and SD of shut periods = 69.358748919 +/- 259.539557773 ms
Range of shut periods from 0.030003175 ms to 6902.281250000 ms
Last shut period = 115.868721008 ms

Number of bursts = 1480
Average length = 6.106142342 ms
Range: 0.039 to 261.102 millisec
Average number of openings= 4.162837838




LOAD FLIP MECHANISM USED Burzomato et al 2004

In [3]:
mecfn = join(datadir, "..", "mec/demomec.mec")
version, meclist, max_mecnum = dcio.mec_get_list(mecfn)
mec = dcio.mec_load(mecfn, meclist[2][0])

10��C9��C8��C7
     �   �   �
    C6��C5��C4
     �   �   �
    O1  O2  O3




Prepare rate constants

In [4]:
rates = [5000.0, 500.0, 2700.0, 2000.0, 800.0, 15000.0, 300.0, 0.1200E+06, 6000.0, 0.4500E+09, 1500.0, 12000.0, 4000.0, 0.9000E+09, 7500.0, 1200.0, 3000.0, 0.4500E+07, 2000.0, 0.9000E+07, 1000, 0.135000E+08]
mec.set_rateconstants(rates)

# Fixed rates.
for i in range(len(mec.Rates)):
    mec.Rates[i].fixed = False

# Constrained rates.
mec.Rates[21].is_constrained = True
mec.Rates[21].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[21].constrain_args = [17, 3]
mec.Rates[19].is_constrained = True
mec.Rates[19].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[19].constrain_args = [17, 2]
mec.Rates[16].is_constrained = True
mec.Rates[16].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[16].constrain_args = [20, 3]
mec.Rates[18].is_constrained = True
mec.Rates[18].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[18].constrain_args = [20, 2]
mec.Rates[8].is_constrained = True
mec.Rates[8].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[8].constrain_args = [12, 1.5]
mec.Rates[13].is_constrained = True
mec.Rates[13].constrain_func = mechanism.constrain_rate_multiple
mec.Rates[13].constrain_args = [9, 2]
mec.update_constrains()

mec.printout(sys.stdout)




class dcpyps.Mechanism
Values of unit rates [1/sec]:
From AF*  	to AF    	alpha1       	5000
From AF  	to AF*    	beta1        	500
From A2F*  	to A2F    	alpha2       	2700
From A2F  	to A2F*    	beta2        	2000
From A3F*  	to A3F    	alpha3       	800
From A3F  	to A3F*    	beta3        	15000
From A3F  	to A3R    	gama3        	300
From A3R  	to A3F    	delta3       	1.2e+05
From A3F  	to A2F    	3kf(-3)      	6000
From A2F  	to A3F    	kf(+3)       	4.5e+08
From A2F  	to A2R    	gama2        	1500
From A2R  	to A2F    	delta2       	12000
From A2F  	to AF    	2kf(-2)      	4000
From AF  	to A2F    	2kf(+2)      	9e+08
From AF  	to AR    	gama1        	7500
From AR  	to AF    	delta1       	1200
From A3R  	to A2R    	3k(-3)       	3000
From A2R  	to A3R    	k(+3)        	4.5e+06
From A2R  	to AR    	2k(-2)       	2000
From AR  	to A2R    	2k(+2)       	9e+06
From AR  	to R    	k(-1)        	1000
From R  	to AR    	3k(+1)       	1.35e+07

Conductance of state AF* (pS)  =      4

In [5]:
theta = np.log(mec.theta())

kwargs = {'nmax': 2, 'xtol': 1e-12, 'rtol': 1e-12, 'itermax': 100,
    'lower_bound': -1e6, 'upper_bound': 0}
likelihood = []

for i in range(len(recs)):
    likelihood.append(Log10Likelihood(bursts[i], mec.kA,
        recs[i].tres, recs[i].tcrit, **kwargs))

In [8]:
def dcprogslik(x, args=None):
    mec.theta_unsqueeze(np.exp(x))
    lik = 0
    for i in range(len(conc)):
        mec.set_eff('c', conc[i])
        lik += -likelihood[i](mec.Q) * math.log(10)
    return lik

iternum = 0
def printiter(theta):
    global iternum
    iternum += 1
    lik = dcprogslik(theta)
    print("iteration # {0:d}; log-lik = {1:.6f}".format(iternum, -lik))
    print(np.exp(theta))

print dcprogslik(theta)
print set_nthreads(1)

-237961.493802
1


In [9]:
%%timeit -n20
dcprogslik(theta)

20 loops, best of 3: 91.3 ms per loop


In [14]:
lik = dcprogslik(theta)
print ("\nStarting likelihood (DCprogs)= {0:.6f}".format(-lik))
start = time.clock()
success = False
result = None
while not success:
    #res = minimize(dcprogslik, np.log(theta), method='Powell', callback=printit, options={'maxiter': 5000, 'disp': True})
    result = minimize(dcprogslik, theta, method='Nelder-Mead', callback=printiter,
        options={'xtol':1e-4, 'ftol':1e-4, 'maxiter': 5000, 'maxfev': 10000,
        'disp': True})
    if result.success:
        success = True
    else:
        theta = result.x
        
end = time.clock()
print ("\nDCPROGS Fitting finished: %4d/%02d/%02d %02d:%02d:%02d\n"
        %time.localtime()[0:6])
print 'time in simplex=', end - start
print '\n\nresult='
print result

print ('\n Final log-likelihood = {0:.6f}'.format(-result.fun))
print ('\n Number of iterations = {0:d}'.format(result.nit))
print ('\n Number of evaluations = {0:d}'.format(result.nfev))
mec.theta_unsqueeze(np.exp(result.x))
print "\n Final rate constants:"
mec.printout(sys.stdout)
print '\n\n'


Starting likelihood (DCprogs)= 237961.493802
iteration # 1; log-lik = 252154.295022
[  5.41561582e+03   5.29996313e+02   2.90758754e+03   2.14771748e+03
   8.51738847e+02   1.64150464e+04   3.16478540e+02   1.33905548e+05
   5.42420858e+08   1.60644964e+03   1.31045940e+04   4.32343868e+03
   8.15436154e+03   1.28247400e+03   9.72503700e+05   1.06690316e+03]
iteration # 2; log-lik = 252154.295022
[  5.41561582e+03   5.29996313e+02   2.90758754e+03   2.14771748e+03
   8.51738847e+02   1.64150464e+04   3.16478540e+02   1.33905548e+05
   5.42420858e+08   1.60644964e+03   1.31045940e+04   4.32343868e+03
   8.15436154e+03   1.28247400e+03   9.72503700e+05   1.06690316e+03]
iteration # 3; log-lik = 252154.295022
[  5.41561582e+03   5.29996313e+02   2.90758754e+03   2.14771748e+03
   8.51738847e+02   1.64150464e+04   3.16478540e+02   1.33905548e+05
   5.42420858e+08   1.60644964e+03   1.31045940e+04   4.32343868e+03
   8.15436154e+03   1.28247400e+03   9.72503700e+05   1.06690316e+03]
iterat

KeyboardInterrupt: 